## Example: load recording

This notebook shows how to load a recording from the labbox_ephys database into a Python object and then use the SpikeInterface library to interact with and view the recording.

In [ ]:
# define some functions

import lbdb
import types
from labbox_ephys import AutoRecordingExtractor
import spikewidgets as sw
import spikeextractors as se
import spiketoolkit as st
import matplotlib.pyplot as plt
def load_recording(recording_id):
    x = lbdb.find(collection='recordings', query=dict(recording_id=recording_id))
    assert len(x) > 0, f'Unable to find recording with id: {recording_id}'
    assert len(x) <= 1, f'Unexpected: found more than one recording with id: {recording_id}'
    doc = x[0]
    R = AutoRecordingExtractor(doc['recording_path'])
    ret = types.SimpleNamespace()
    ret.document = doc
    ret.extractor = R
    return ret

In [ ]:
# Load the recording by ID (you can get the idea from the recordings table)
recording_id = 'sf:paired_kampff/2014_11_25_Pair_3_0'
R = load_recording(recording_id).extractor

In [ ]:
# print information from the recording
sampling_freq = R.get_sampling_frequency()
channel_ids = R.get_channel_ids()
duration_sec = R.get_num_frames() / R.get_sampling_frequency()
print(f'{len(channel_ids)} channels; {sampling_freq} Hz; {duration_sec} sec')

In [ ]:
# Get a sub-recording (60 seconds of a subset of channels)
R2 = se.SubRecordingExtractor(parent_recording=R, start_frame=0, end_frame=sampling_freq*60, channel_ids=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [ ]:
# Filter the recording (lazy execution)
R2f = st.preprocessing.bandpass_filter(recording=R2, freq_min=300, freq_max=6000, freq_wid=1000)

In [ ]:
# Plot a portion of the recording and the filtered recording
plt.figure(figsize=(16,5))
sw.TimeseriesWidget(recording=R2, trange=(1.3, 1.4), ax=plt.gca()).plot()
plt.title(f'{recording_id} - raw');

plt.figure(figsize=(16,5))
sw.TimeseriesWidget(recording=R2f, trange=(1.3, 1.4), ax=plt.gca()).plot()
plt.title(f'{recording_id} - filtered');